## 1. Solve UR IK with OptModel

In [1]:
import os

import numpy as np
from RVBUST.RPS import *
np.set_printoptions(precision=4, suppress=True)

In [2]:
robot_model = LoadRobotModelFromFile(GetRobotFile("UniversalRobots", "UR5"))
manipulator = robot_model.GetActiveManipulator()
kin_solver = CreateKinSolver(manipulator)
dof = 6
joints_bounds = np.tile(np.pi, dof)

### 1.1 Define problem cost and constraints

In [3]:
class IKProb1:
    def __init__(self, kin_solver: URsKinematics):
        self.kin_solver = kin_solver
        self.target_pose = SE3()
    
    def SetTargetPose(self, pose: SE3):
        self.target_pose = pose
    
    def GetCostVaue(self, x: np.ndarray, *args)->float:
        pose = self.kin_solver.GetPositionFK(Rx(x))[1]
        return (pose - self.target_pose).WeightedNorm()
    # there is no constraints
    
ikprob1 = IKProb1(kin_solver)

### 1.2 Setup OptModel

In [4]:
model: OptModel = OptModel()
variable_set = VariableSet(dof, bounds=[Bounds(-b, b) for b in joints_bounds], name="joints")
model.SetVariableSet(variable_set)

In [5]:
cost_term = ExCost(ikprob1.GetCostVaue)
model.AddCostTerm(cost_term)

### 1.3 Solve with OptSolver

In [6]:
solver = NloptSolver()

#### Make a guess of initial variable values

In [7]:
# joints = np.array([0, -np.pi, np.pi/2, 0, np.pi, 0])
joints = np.random.uniform(-np.pi, np.pi, 6)
_, target_pose = kin_solver.GetPositionFK(Rx(joints))
ikprob1.SetTargetPose(target_pose)
joints_init = joints + np.random.uniform(-0.5, 0.5, 6) 
# joints_init = np.random.uniform(-np.pi, np.pi, 6)
np.clip(joints_init, -joints_bounds+0.001, joints_bounds-0.001)

array([-1.1546,  2.555 ,  1.987 ,  1.6995,  1.2692,  2.5289])

In [8]:
joints, target_pose

(array([-0.8969,  2.98  ,  2.4147,  1.2743,  1.7175,  2.3291]),
 SE3: [-0.006669, 0.163989, 0.20687, -0.271672, 0.357191, -0.378893, 0.809351])

In [9]:
joints_init

array([-1.1546,  2.555 ,  1.987 ,  1.6995,  1.2692,  2.5289])

#### Set solver options, eg. stop criteria

In [10]:
solver.Initialize(joints_init, NloptAlgorithm.LD_SLSQP)
# opt = solver.GetNloptApp()
# opt.set_xtol_rel(1e-5)

True

In [11]:
model.GetCostValue(joints_init)

0.474950993119303

In [12]:
solver.Solve(model)

True

#### Verify

In [13]:
joints_opt = model.GetVariableValue()
model.GetCostValue(joints_opt), joints_opt

(3.497103690599789e-06,
 array([-0.8969,  2.98  ,  2.4147,  1.2743,  1.7175,  2.3291]))

In [14]:
kin_solver.GetPositionFK(Rx(joints_opt))[1] - ikprob1.target_pose

SE3Tangent: [-4.59874e-07, 1.74908e-06, 1.91157e-06, -9.753e-07, -7.85275e-07, -2.45163e-06]